In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

c:\Users\BI\anaconda3\lib\site-packages\requests\__init__.py:78: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "


In [2]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [4]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. c:\Users\BI\Desktop\KoBert\KoBERT-master\.cache\kobert_v1.zip
using cached model. c:\Users\BI\Desktop\KoBert\KoBERT-master\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


!pip install torch

In [5]:
device = torch.device('cpu')

In [6]:
model = torch.load('C:/Users/BI/Desktop/KoBert/KoBERT-master/c.pth',map_location=device)

In [7]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [8]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [9]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")
    return np.argmax(logits)

using cached model. c:\Users\BI\Desktop\KoBert\KoBERT-master\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
import socket, threading
def binder(client_socket, addr):
    print('Connected by', addr);
    try:
        while True:
            data = client_socket.recv(4);
            length = int.from_bytes(data, "big");
            data = client_socket.recv(length);
            msg = data.decode();
            print('Received from', addr, msg);
            msg = predict(msg)
            msg = str(msg)
            print(msg)
            data = msg.encode();
            length = len(data);
            client_socket.sendall(length.to_bytes(4, byteorder='big'))
            client_socket.sendall(data);
    except:
        print("except : " , addr);
    finally:
        client_socket.close();

server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM);
server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1);
server_socket.bind(('', 9999));
server_socket.listen();
try:
    while True:
        client_socket, addr = server_socket.accept();
        th = threading.Thread(target=binder, args = (client_socket,addr));
        th.start();
except:
    print("server");
finally:
    server_socket.close();

Connected by ('127.0.0.1', 58028)
Received from ('127.0.0.1', 58028) 아 진짜열받는다
2
Received from ('127.0.0.1', 58028) 
1
except :  ('127.0.0.1', 58028)
Connected by ('127.0.0.1', 58032)
Received from ('127.0.0.1', 58032) 아 진짜열받는다
2
Received from ('127.0.0.1', 58032) 
1
except :  ('127.0.0.1', 58032)
Connected by ('127.0.0.1', 58041)
Received from ('127.0.0.1', 58041) 
1
Received from ('127.0.0.1', 58041) 
1
except :  ('127.0.0.1', 58041)
Connected by ('127.0.0.1', 58042)
Received from ('127.0.0.1', 58042) zzzzzzzzzz
4
Received from ('127.0.0.1', 58042) 
1
except :  ('127.0.0.1', 58042)
Connected by ('127.0.0.1', 58043)
Received from ('127.0.0.1', 58043) 아 진짜 열받는다..
2
Received from ('127.0.0.1', 58043) 
1
except :  ('127.0.0.1', 58043)
Connected by ('127.0.0.1', 58045)
Received from ('127.0.0.1', 58045) 아 진짜 열받는다;;
2
Received from ('127.0.0.1', 58045) 
1
except :  ('127.0.0.1', 58045)
Connected by ('127.0.0.1', 58313)
Received from ('127.0.0.1', 58313) 아 진짜 열받는다;;
2
Received from ('127.0.0

In [1]:
process=[
    ['P1', 8, 3],
    ['P2', 2, 1],
    ['P3', 1, 9],
    ['P4', 3, 2],
    ['P5', 4, 4]
]
res=[]

prirun = 0
run = 0



for a in range(0, len(process)) :
    res.append([])
    res[0].append(process[a][0])
    run += prirun
    arrive = process[a][1]
    waittime=run-arrive
    res.append([])
    res[1].append(waittime)
    returntime = waittime + process[a][2]
    res.append([])
    res[2].append(returntime)
    prirun = process[a][2]


for p in range(0, len(process)) :
    print(res[0][p], "대기시간 : ", res[1][p] ,"\t", "반환시간 : ", res[2][p])

P1 대기시간 :  -8 	 반환시간 :  -5
P2 대기시간 :  1 	 반환시간 :  2
P3 대기시간 :  3 	 반환시간 :  12
P4 대기시간 :  10 	 반환시간 :  12
P5 대기시간 :  11 	 반환시간 :  15


In [8]:
bt=[]
print("Enter the number of process: ")
n=int(input())
print("Enter the burst time of the processes: \n")
bt=list(map(int, input().split()))

wt=[]
avgwt=0
tat=[]
avgtat=0

wt.insert(0,0)
tat.insert(0,bt[0])

for i in range(1,len(bt)):
   wt.insert(i,wt[i-1]+bt[i-1])
   tat.insert(i,wt[i]+bt[i])
   avgwt+=wt[i]
   avgtat+=tat[i]

avgwt=float(avgwt)/n
avgtat=float(avgtat)/n
print("\n")
print("Process\t  Burst Time\t  Waiting Time\t  Turn Around Time")

for i in range(0,n):
   print(str(i)+"\t\t"+str(bt[i])+"\t\t"+str(wt[i])+"\t\t"+str(tat[i]))
   print("\n")

print("Average Waiting time is: "+str(avgwt))
print("Average Turn Arount Time is: "+str(avgtat))

Enter the number of process: 
Enter the burst time of the processes: 



Process	  Burst Time	  Waiting Time	  Turn Around Time
0		1		0		1


1		8		1		9


2		3		9		12




IndexError: list index out of range

In [10]:
class FCFS:
    def processData(self, no_of_processes):
        process_data = []
        for i in range(no_of_processes):
            temporary = []
            process_id = int(input("Enter Process ID: "))

            arrival_time = int(input("Enter Arrival Time: "))

            burst_time = int(input(f"Enter Burst Time for Process {process_id}: "))

            temporary.extend([process_id, arrival_time, burst_time])
            process_data.append(temporary)
        FCFS.schedulingProcess(self, process_data)

    def schedulingProcess(self, process_data):
        process_data.sort(key=lambda x: x[1])
       '''
       Sort according to Arrival Time 
       '''
        start_time = []
        exit_time = []
        s_time = 0
        for i in range(len(process_data)):
            if s_time < process_data[i][1]:
                s_time = process_data[i][1]
            start_time.append(s_time)
            s_time = s_time + process_data[i][2]
            e_time = s_time
            exit_time.append(e_time)
            process_data[i].append(e_time)
        t_time = FCFS.calculateTurnaroundTime(self, process_data)
        w_time = FCFS.calculateWaitingTime(self, process_data)
        FCFS.printData(self, process_data, t_time, w_time)

    def calculateTurnaroundTime(self, process_data):
        total_turnaround_time = 0
        for i in range(len(process_data)):
            turnaround_time = process_data[i][3] - process_data[i][1]
            '''
            turnaround_time = completion_time - arrival_time
            '''
            total_turnaround_time = total_turnaround_time + turnaround_time
            process_data[i].append(turnaround_time)
        average_turnaround_time = total_turnaround_time / len(process_data)
        '''
        average_turnaround_time = total_turnaround_time / no_of_processes
        '''
        return average_turnaround_time

    def calculateWaitingTime(self, process_data):
        total_waiting_time = 0
        for i in range(len(process_data)):
            waiting_time = process_data[i][4] - process_data[i][2]
            '''
            waiting_time = turnaround_time - burst_time
            '''
            total_waiting_time = total_waiting_time + waiting_time
            process_data[i].append(waiting_time)
        average_waiting_time = total_waiting_time / len(process_data)
        '''
        average_waiting_time = total_waiting_time / no_of_processes
        '''
        return average_waiting_time

    def printData(self, process_data, average_turnaround_time, average_waiting_time):

        print("Process_ID  Arrival_Time  Burst_Time  Completion_Time  Turnaround_Time  Waiting_Time")

        for i in range(len(process_data)):
            for j in range(len(process_data[i])):
                print(process_data[i][j], end="				")
            print()

        print(f'Average Turnaround Time: {average_turnaround_time}')

        print(f'Average Waiting Time: {average_waiting_time}')

if __name__ == "__main__":

    no_of_processes = int(input("Enter number of processes: "))

    fcfs = FCFS()
    fcfs.processData(no_of_processes)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 18)